<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    TeX: {
      Macros: {
        ket: ["\\left| #1 \\right\\rangle", 1],
        bra: ["\\left\\langle #1 \\right|", 1],
        braket: ["\\left\\langle #1 \\middle| #2 \\right\\rangle", 2]
      }
    }
  });
</script>

# TP1 - Premiers circuits quantiques (Paires de Bell et Téléportation)

In [ ]:
# only if using Google Colab:
!pip install myqlm

In [ ]:
import numpy as np

from qat.lang import QRoutine, H, CNOT, RY, Z, X, Program
from qat.qpus import get_default_qpu

qpu = get_default_qpu()

def display_result(circuit, nbshots=0, idx=None):
    result = qpu.submit(circuit.to_job(nbshots=nbshots, qubits=idx))
    if nbshots:
        tmp = {}
        for sample in result:
            state = sample.state
            if not state in tmp:
                tmp[state] = 0.
            tmp[sample.state] += sample.probability
        for state, proba in tmp.items():
            print("Etat %s: probabilité %s" % (state, proba))
    else:
        for sample in result:
            print("Etat %s: probabilité %s, amplitude %s" % (sample.state, sample.probability, sample.amplitude))

## Rappels de base sur les circuits quantiques

L'objectif de cette première partie du TP est de se familiariser avec le concept de construction de circuits quantiques. Pour créer des circuits quantiques et lancer des simulations nous allons utiliser **myQLM** dans un environnement Python. La documentation utilisateur du package est disponible à l'adresse [https://myqlm.github.io](https://myqlm.github.io).

Visuellement, un circuit quantique est constitué de plusieurs fils horizontaux qui représentent les différents **qubits** qui évoluent dans le **temps** (de la gauche vers la droite). Au cours de l'évolution du temps, des **portes** quantiques sont appliquées sur un ou plusieurs qubits afin de leur appliquer une transformation.

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/circuit0.png)

### Transformations unitaires

Ces transformations sont dites **unitaires** afin d'assurer deux principes importants en mécanique quantique qui sont :

- **Réversibilité** des opérations
- Conservation d'une **norme 1**

Un opérateur $U$ est dit unitaire si

$$
U^\dagger U = U U^\dagger = I.
$$

### Circuit quantique $\longleftrightarrow$ vecteurs / matrices

On peut très bien passer du formalisme circuit quantique à celui des vecteurs et matrices :

- Système à $n$ qubits --> vecteur de taille $2^n$
- Opérateur sur $n$ qubits --> matrice de taille $2^n \times 2^n$

### Portes quantiques de base
Nous allons passer en revue quelques portes quantiques de base qui vous serviront pour la suite.

**Porte X : NOT quantique**

$$
X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
$$

La porte $X$ est une base importante pour construire des circuits quantiques puisqu'elle applique l'opérateur NOT sur un qubit. La transformation effectuée est :

- $\left| 0 \right \rangle$ --> $\left| 1 \right \rangle$
-  $\left| 1 \right \rangle$ --> $\left| 0 \right \rangle$

*Dans myqlm : X(q0)*

**Porte H : création de superposition**

$$
H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
$$


La porte $H$ est utile pour créer des superpositions quantiques (uniformes). La transformation effectuée est :

- $\left| 0 \right \rangle$ --> $\frac{1}{\sqrt{2}} (\left| 0 \right \rangle + \left| 1 \right \rangle)$
- $\left| 1 \right \rangle$ --> $\frac{1}{\sqrt{2}} (\left| 0 \right \rangle - \left| 1 \right \rangle)$

ce qui permet bien d'obtenir une superposition uniforme. Notons que pour le cas $\left| 1 \right \rangle$ --> $\frac{1}{\sqrt{2}} (\left| 0 \right \rangle - \left| 1 \right \rangle)$, un $-$ apparaît devant le $\left| 1 \right \rangle$, on introduit ce qu'on appelle une **phase relative**.

*Dans myqlm : H(q0)*

**Portes CNOT : création d'intrication**

$$
CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0  \end{pmatrix}
$$

La porte CNOT est une porte qui agit sur 2 qubits, en pratique c'est une porte X contrôlée par le premier qubit, appliquée sur le second qubit. Cela permet de créer de l'intrication entre les deux qubits puisqu'il y a une dépendance entre les états des deux qubits entre eux. Si le premier qubit est à 0 alors le second ne subit aucun changement.

- $\left| 0 \right \rangle \left| 0 \right \rangle$ --> $\left| 0 \right \rangle \left| 0 \right \rangle$
- $\left| 0 \right \rangle \left| 1 \right \rangle$ --> $\left| 0 \right \rangle \left| 1 \right \rangle$

Au contraire, si le premier qubit est à 1 alors l'opération NOT est appliquée sur le second qubit.

- $\left| 1 \right \rangle \left| 0 \right \rangle$ --> $\left| 1 \right \rangle \left| 1 \right \rangle$
- $\left| 1 \right \rangle \left| 1 \right \rangle$ --> $\left| 1 \right \rangle \left| 0 \right \rangle$

Dans la représentation circuit, une porte controlée est représentée de la façon suivante

![](https://raw.githubusercontent.com/thomastuloup/quantum_labs_polytech_saclay/main/TPs/img/control_gates.png)

De gauche à droite : CNOT représentée comme une C-X (X contrôlée), représentation usuelle de la CNOT, X multi-contrôlée. Le point noir correspond au qubit qui a un rôle de contrôle.

*Dans myqlm : CNOT(q0, q1) ou X.ctrl(1)(q0, q1) ou X.ctrl(2)(q0, q1, q2) ...*

**Portes rotations RX, RY et RZ : portes paramétrées**

$$
RX(2\theta) = \begin{pmatrix} cos(\theta) & -i sin(\theta) \\ -i sin(\theta) & cos(\theta) \end{pmatrix}, \; \; \;
RY(2\theta) = \begin{pmatrix} cos(\theta) & - sin(\theta) \\ sin(\theta) & cos(\theta) \end{pmatrix}, \; \; \;
RZ(2\theta) = \begin{pmatrix} e^{-i\theta} & 0 \\ 0 & e^{i\theta} \end{pmatrix}, 
$$

Certaines portes peuvent prendre un paramètre. C'est le cas des portes de rotation RX, RY et RZ qui prennent en paramètre un angle $\theta$. Elles permettent notamment de créer des superpositions plus complexes qu'une simple superposition uniforme offerte par la porte H.

*Dans myqlm : RX(2\*theta)(q0)*

# Exercice 1 : les paires de Bell

Nous allons construire notre premier circuit quantique avec myQLM et faire des petites expériences avec. En informatique quantique les états de Bell sont des états d'intrication maximale entre deux qubits
- $\left| \Phi^+_= \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 0 \right \rangle + \left| 1 \right \rangle \left| 1 \right \rangle)$
- $\left| \Phi^-_= \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 0 \right \rangle - \left| 1 \right \rangle \left| 1\right \rangle)$
- $\left| \Phi^+_{\neq} \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 1 \right \rangle + \left| 1 \right \rangle \left| 0 \right \rangle)$
- $\left| \Phi^-_{\neq} \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 1 \right \rangle - \left| 1 \right \rangle \left| 0 \right \rangle)$

Peu importe la paire de qubits qu'on choisi, si l'on mesure le premier qubit alors on sait immédiatement dans quel état sera mesuré le second. On va commencer par créer un circuit quantique qui prépare l'état de Bell 

$$\ket{\Phi^+_=} = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 0 \right \rangle + \left| 1 \right \rangle \left| 1 \right \rangle)$$

In [ ]:
def bell_pair_1() :
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine de 2 qubits
    qubits = rout.new_wires(2)

    # Créer une superposition uniforme sur le premier qubit à l'aide d'une porte H
    H(qubits[0])

    # Intriquer le premier qubit avec le second qubit à l'aide d'une porte CNOT
    CNOT(qubits[0], qubits[1])

    return rout

Le code ci-dessus crée une fonction *bell_pair_1* qui va créer une routine quantique. La routine quantique contient ici 2 *qubits* et applique une porte H puis une porte CNOT. On peut afficher le circuit quantique généré à l'aide de la fonction *display*

In [ ]:
bell_pair_1().display()

Maintenant que notre circuit quantique est construit nous allons vérifier à l'aide d'une simulation qu'il fait effectivement bien ce que l'on veut.

In [ ]:
# Affiche les resultats
display_result(bell_pair_1())

On obtient bien l'état $\left| \Phi^+_= \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 0 \right \rangle + \left| 1 \right \rangle \left| 1 \right \rangle)$ quand on regarde les amplitudes des résultats. Maintenant, nous allons nous entrainer à la création de circuit en créant les autres paires de Bell.

**Question 1 : Implémenter le circuit quantique pour l'état**
$$\left| \Phi^+_{\neq} \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 1 \right \rangle + \left| 1 \right \rangle \left| 0 \right \rangle)$$

L'état quantique ici est très similaire au précédent, la seule différence est qu'on veut que dans la paire les qubits soient différents plutôt que de même valeur (0 avec 1 et 1 avec 0, au lieu de 0 avec 0 et 1 avec 1). Pour ce faire, on conserve le même circuit que pour *bell_pair_1*, qui nous donne en sortie 00 ou 11 et il suffit d'inverser la valeur du deuxième qubit pour avoir 01 ou 10. 

In [ ]:
def bell_pair_2() :
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine de 2 qubits
    qubits = rout.new_wires(2)

    # Créer une superposition uniforme sur le premier qubit à l'aide d'une porte H
    H(qubits[0])

    # Intriquer le premier qubit avec le second qubit à l'aide d'une porte CNOT
    CNOT(qubits[0], qubits[1])

    # Flipper la valeur du second qubit avec une porte X (NOT) 
    X(qubits[1])

    return rout

On vérifie ensuite qu'on obtient le bon résultat en sortie

In [ ]:
# Affiche les resultats
display_result(bell_pair_2())

**Question 2 : Implémententer le circuit quantique pour l'état**
$$\left| \Phi^-_= \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 0 \right \rangle - \left| 1 \right \rangle \left| 1\right \rangle)$$

Là on se retrouve sur une paire de valeurs identiques (00 et 11) mais une phase (le - au lieu du +) rend cet état différent. Pour introduire cette phase on peut utiliser la porte Hadamard sur le premier qubit dans l'état $\ket{1}$. En effet, quand on regarde la matrice
$$
H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
$$

Dans la deuxième colonne, on a les valeurs 1 et -1, ce qui signifie bien que $\left| 1 \right \rangle \rightarrow \frac{1}{\sqrt{2}}( \left| 0 \right \rangle - \left| 1 \right \rangle)$. On introduit ici la phase et ensuite il suffit de copier la valeur du qubit 1 dans le qubit 2 avec une porte CNOT, comme pour *bell_pair_1*.


In [ ]:
def bell_pair_3() :
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine de 2 qubits
    qubits = rout.new_wires(2)

    # Appliquer une porte X sur le premier qubit pour le mettre dans l'état 1
    X(qubits[0])

    # Créer une superposition uniforme sur le premier qubit à l'aide d'une porte H
    H(qubits[0])

    # Intriquer le premier qubit avec le second qubit à l'aide d'une porte CNOT
    CNOT(qubits[0], qubits[1])

    return rout

On vérifie ensuite qu'on obtient le bon résultat en sortie

In [ ]:
# Affiche les resultats
display_result(bell_pair_3())

**Question 3 : Implémenter le circuit quantique pour l'état**
$$\left| \Phi^-_{\neq} \right \rangle = \frac{1}{\sqrt{2}} (\left| 0 \right \rangle \left| 1 \right \rangle - \left| 1 \right \rangle \left| 0 \right \rangle)$$

Ici, on combine les deux modifications précédentes pour obtenir cet état quantique, avec des paires de valeurs différentes ET une phase relative.

In [ ]:
def bell_pair_4() :
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine de 2 qubits
    qubits = rout.new_wires(2)

    # Circuit complet à inventer
    X(qubits[0])
    H(qubits[0])
    CNOT(qubits[0], qubits[1])
    X(qubits[1])

    return rout

On vérifie ensuite qu'on obtient le bon résultat en sortie

In [ ]:
display_result(bell_pair_4())

# Exercice 2 : téléportation quantique

La téléportation quantique est une technique permettant de transmettre de l'information à distance. Elle consiste à transférer l'état quantique d'un système A vers un système B en se reposant sur l'intrication quantique entre les deux systèmes. Imaginons qu'Armando a un qubit dans l'état $\left| \psi \right \rangle_A = \alpha \left| 0 \right \rangle_A + \beta \left| 1 \right \rangle_A$ et souhaite le transmettre à Bridget. Il pourrait directement communiquer $\alpha$ et $\beta$ via un canal classique à Bridget mais il est possible de faire mieux.

Supposons qu'en amont Armando et Bridget se sont partagé une paire de qubits maximalement intriqués $\ket{\phi}_{ab} = \frac{1}{\sqrt{2}}(\left| 0 \right \rangle_a \left| 0 \right \rangle_b + \left| 1 \right \rangle_a \left| 1 \right \rangle_b)$, Armando conserve le qubit $a$ tandis que Bridget emporte avec elle le qubit $b$. L'état initial du système à 3 qubits (le qubit d'Armando et les deux qubits de la paire) est donc :

$$
\frac{1}{\sqrt{2}} \left| \psi \right \rangle_A(\left| 0 \right \rangle_a \left| 0 \right \rangle_b + \left| 1 \right \rangle_a \left| 1 \right \rangle_b)
$$

**Question 1 : Supposons $\alpha$ et $\beta$ réels. Sachant que la porte RY($\theta$) correspond à l'opération matricielle**

$$RY(2\theta) = \begin{pmatrix} cos(\theta) & - sin(\theta) \\ sin(\theta) & cos(\theta) \end{pmatrix}$$

**Quelle valeur doit prendre $\theta$ pour que le circuit ci dessous réalise l'opération $\left| 0 \right \rangle \rightarrow \alpha \left| 0 \right \rangle + \beta \left| 1 \right \rangle$**


In [ ]:
def state_prep(alpha, beta):
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine de 1 qubit
    qubits = rout.new_wires(1)

    theta = np.sign(beta) *np.arccos(alpha)  # Remplacer la valeur de theta

    # Rotation préparant la superposition
    RY(2.*theta)(qubits[0])

    return rout

On peut tester notre préparation d'état avec le code ci-dessous

In [ ]:
alpha = 0.866
beta = -0.5

display_result(state_prep(alpha, beta))

**Question 2 : Créer le circuit quantique de l'initialisation du protocole de téléportation quantique qui prépare l'état**

$$
\frac{1}{\sqrt{2}} \left| \psi \right \rangle_A (\left| 0 \right \rangle_a \left| 0 \right \rangle_b + \left| 1 \right \rangle_a \left| 1 \right \rangle_b)
$$

Pour ce faire, on peut utiliser la fonction *state_prep* pour préparer le qubit $A$ et la fonction *bell_pair_1* de l'exercice précédent pour la paire de qubits $ab$

In [ ]:
def initialisation(alpha, beta):
    # Création d'une routine quantique
    rout = QRoutine()
    # Création dans la routine du qubit A
    qubit_A = rout.new_wires(1)
    # Création dans la routine de la paire de qubits ab
    qubits_ab = rout.new_wires(2)

    # Prépare l'état du qubit A
    state_prep(alpha, beta)(qubit_A)

    # Prépare l'intrication sur la paire ab
    bell_pair_1()(qubits_ab)

    return rout

On peut tester notre initialisation avec le code ci-dessous. Les seuls états atteignables sont $\left| 000 \right \rangle$, $\left| 011 \right \rangle$, $\left| 100 \right \rangle$ et $\left| 111 \right \rangle$.

In [ ]:
# Affiche les resultats
display_result(initialisation(alpha, beta))

Maintenant qu'Armando et Bridget sont à distance, Armando va faire interagir son qubit $A$ avec son qubit de la paire $a$ à l'aide du porte CNOT. L'objectif est de faire partager l'information contenue dans l'état $\ket{\psi}$ au qubit $a$. L'état quantique du système après cette opération est

$$
\frac{\alpha}{\sqrt{2}}\left| 0 \right \rangle_A (\left| 00 \right \rangle_{ab}+\left| 11 \right \rangle_{ab}) + \frac{\beta}{\sqrt{2}}\left| 1 \right \rangle_A (\left| 10 \right \rangle_{ab}+\left| 01 \right \rangle_{ab}).
$$

Ensuite, Armando va appliquer une porte H sur le qubit $A$ ce qui donne l'état quantique

$$
\begin{aligned}
      & \frac{1}{2} \left| 0 \right \rangle_A \left| 0 \right \rangle_a \;\;\; (\alpha \left| 0 \right \rangle_b + \beta \left| 1 \right \rangle_b) \\
    + & \frac{1}{2} \left| 0 \right \rangle_A \left| 1 \right \rangle_a \;\;\; (\beta \left| 0 \right \rangle_b + \alpha \left| 1 \right \rangle_b) \\
    + & \frac{1}{2} \left| 1 \right \rangle_A \left| 0 \right \rangle_a \;\;\; (\alpha \left| 0 \right \rangle_b - \beta \left| 1 \right \rangle_b) \\
    - & \frac{1}{2} \left| 1 \right \rangle_A \left| 1 \right \rangle_a \;\;\; (\beta \left| 0 \right \rangle_b - \alpha \left| 1 \right \rangle_b)
\end{aligned}
$$

**Question 3 : Créer le circuit quantique qui permet de réaliser cette opération intermédiaire**

In [ ]:
def operation_intermediaire():
    # Création d'une routine quantique
    rout = QRoutine()

    # Création dans la routine du qubit A
    qubit_A = rout.new_wires(1)

    # Création dans la routine de la paire de qubit ab
    qubits_ab = rout.new_wires(2)

    # Opération intermédiaire
    CNOT(qubit_A, qubits_ab[0])
    H(qubit_A)

    return rout

On remarque que l'état $\left| \psi \right \rangle_A = \alpha \left| 0 \right \rangle_A + \beta \left| 1 \right \rangle_A$ a été plus ou moins transmis au qubit $b$. En réalité il a été parfaitement transmis dans 1 cas, quand les qubits $A$ et $a$ sont dans l'état $\left| 0 \right \rangle_A \left| 0 \right \rangle_a$. Dans les 3 autres cas, il faut effectuer une modification sur le qubit de Bridget $b$ en fonction de ce qui est mesuré par Armando sur ces deux qubits.
- Qubit $A$ mesuré à 1 $\longrightarrow$ porte Z sur le qubit $b$
- Qubit $a$ mesuré à 1 $\longrightarrow$ porte X sur le qubit $b$

Il est désormais temps de tout assembler et de rajouter cette dernière modification. Pour ce faire, nous allons créer un programme quantique, qui permet d'allier bits quantiques et bits classiques (et donc de faire des mesures).

In [ ]:
def teleportation(alpha, beta):
    # Création d'un Programme quantique
    prog = Program()

    # Allocation des 3 qubits
    qubits = prog.qalloc(3)
    # Allocation des 2 bits classiques pour les mesures
    bits = prog.calloc(2)

    #Initialisation de la téléportation
    initialisation(alpha, beta)(qubits)
    #Opérations intermédiaires d'Armando
    operation_intermediaire()(qubits)

    # Mesure par Armando de ses deux qubits
    prog.measure(qubits[0:2], bits)

    # Modification suivant le bit_A du qubit b
    prog.cc_apply(bits[0], Z, qubits[2])  # Controle classique avec le bits[0] de la porte Z appliquée sur le qubit b
    
    # Modfication suivant le bit_a du qubit b
    prog.cc_apply(bits[1], X, qubits[2])  # Controle classique avec le bits[1] de la porte Z appliquée sur le qubit b
    
    return prog

In [ ]:
circuit = teleportation(alpha, beta).to_circ()
circuit.display()

On va ensuite lancer une simulation de notre téléportation quantique. Ici, comme nous effectuons des mesures intermédiaires dans notre circuit quantique, nous devons fixer un nombre de shots pour le simulation et nous obtiendrons seulement des probabilités en sortie (et non des amplitudes). Regardons donc les probabilités en sortie d'obtenir 0 et 1 sur notre qubit b uniquement. 

In [ ]:
display_result(circuit, nbshots=1000, idx=[2])

On obtient les mêmes probabilités que nous avions lors de l'initialisation du qubit $A$ dans l'état $\left| \psi \right \rangle_A$ (voir plus haut). On a donc réussi à téléporter l'état du qubit $A$ dans le qubit $b$ en utilisant seulement deux bits classiques d'information !

# Pour aller plus loin

- L'intrication est un phénomène quantique très particulier qui est inexplicable par des modèles classiques. Lorsque deux qubits sont intriqués, il est impossible d'exprimer leur état de manière séparée, nous sommes forcés de considérer l'état joint de ces deux qubits. [L'expérience d'Alain Aspect](https://fr.wikipedia.org/wiki/Exp%C3%A9rience_d%27Aspect) met en lumière ce phénomène et prouve la violation des [inégalités de Bell](https://fr.wikipedia.org/wiki/In%C3%A9galit%C3%A9s_de_Bell).
- La téléportation quantique ne viole pas la théorie de la relativité restreinte, surtout l'idée que l'information ne peut pas être transmise à une vitesse supérieures à celle de la lumière dans le vide. En effet, Armando doit transmettre 2 bits classiques d'information à Bridget s'il veut transmettre l'information contenue dans son qubit. On transmet donc plus d'information mais à une vitesse toujours limitées par les lois de la relativité.
- En informatique quantique, il existe [le théorème de non-clonage](https://fr.wikipedia.org/wiki/Non-clonage_quantique). Ce théorème explique qu'on ne peut pas cloner un état quantique inconnu et arbitraire. La copie parfaite est donc impossible en informatique quantique, mais l'intrication permet de faire partager de l'information entre plusieurs systèmes quantiques. Dans le cas de la téléportation quantique, on ne viole pas ce théorème puisqu'à la fin de l'exprience le qubit en possession de Armando ne contient plus l'information initiale. On a alors une transmission et non un clonage.